In [1]:
from qdrant_client import QdrantClient, models
from app.config import settings as config
from app.utils.utils import split_docs
from app.web_loader.bs_loader import load_web_docs, urls
from app.db.vector_store import VectorStore

In [2]:
client = QdrantClient(config.qdrant_url, api_key=config.qdrant_api_key)

In [3]:
client.create_collection(
    collection_name="sutd",
    vectors_config=models.VectorParams(
        size=config.embeddings_dim, distance=models.Distance.COSINE
    ),
)

True

In [4]:
# client.delete_collection(collection_name="sutd")

In [5]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='sutd')])

In [6]:
docs = load_web_docs(urls)
chunks = split_docs(docs)

In [7]:
print(chunks[4])

Document(text='ousing Sports and recreation centre F&B and services Library About About OVERVIEW Design AI At a glance Diversity and inclusion Sustainability PEOPLE Leadership Board of Trustees President Emeritus Professorships Faculty MORE Happenings Careers at SUTD Partnering with SUTD On this page Popular searches quick links Discover SUTD The world’s first Design AI university Innovation @ SUTD AirXeed radiosonde AI @ SUTD Mustango – DALL-E for music Discover SUTD The world’s first Design AI university Pla', metadata={'source': 'https://www.sutd.edu.sg/'})


In [8]:
vector_store = VectorStore(config)
document_ids = vector_store.add_documents(docs=chunks)

# print(document_ids[:3])

In [9]:
collection_info = vector_store.client.get_collection(vector_store.collection_name)
print(f"Collection info: {collection_info}")

# Count the number of points in the collection
point_count = vector_store.client.count(vector_store.collection_name)
print(f"Number of documents in vector store: {point_count}")

Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=975 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantiz

In [10]:
query = "what is sutd?"
query_embedding = vector_store.get_embeddings(query)
results = vector_store.client.query_points(
    collection_name=vector_store.collection_name,
    query=query_embedding,
    limit=5,
)
print(results)

points=[ScoredPoint(id=752, version=0, score=0.74369144, payload={'text': 'About - Singapore University of Technology and Design (SUTD) About Us | SUTD Academic calendar Alumni Faculty Partner with SUTD Resources Design AI Education Design AI Education STUDY Undergraduate Graduate Continuing education (SUTD Academy) 42 Singapore MORE Library Accreditation Cyber-Physical Learning Alliance Undergraduate courses PILLARS, CLUSTERS & PROGRAMMES ASD Architecture and Sustainable Design DAI Design and Artificial Intelligence EPD Engineering Product Development ESD Engineering ', 'metadata': {'source': 'https://www.sutd.edu.sg/about'}}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=0, version=0, score=0.7236957, payload={'text': 'Home - Singapore University of Technology and Design (SUTD) Singapore University of Technology and Design | SUTD Academic calendar Alumni Faculty Partner with SUTD Resources Design AI Education Design AI Education STUDY Undergraduate Graduate Continuin